In [2]:
import numpy as np
import pandas as pd

In [125]:
df_test = pd.read_csv("./data_10k.csv", sep=',')

In [195]:
df_test.shape

(10000, 9663)

In [196]:
df = df_test.set_index("id")

In [197]:
a = pd.DataFrame(df.sum()).sort_values(by=0, ascending=False)
b = pd.DataFrame(df.sum(axis=1)).sort_values(by=0, ascending=False)

In [198]:
# translation_recipe_id_index = df_test['id'].reset_index().set_index('id').to_dict()['index']

In [199]:
columns = a[a[0] > 1].index.values

In [200]:
a[a[0] > 1][:30]

,0
salt,3754.0
sugar,3081.0
onion,2819.0
egg,2282.0
butter,2229.0
flour,1999.0
water,1753.0
pepper,1746.0
olive oil,1456.0
milk,1456.0


In [201]:
df['pepper'] = df['pepper'] + df['ground pepper']
df.drop(columns=['ground pepper'], inplace=True)
columns = a[a[0] > 1].index.values
drop_columns = ["salt ground pepper", "half", "salt pepper", "half"]
cols = np.setdiff1d(columns, drop_columns)

In [202]:
a = pd.DataFrame(df.sum()).sort_values(by=0, ascending=False)

In [203]:
a[a[0] > 1][:30]

,0
salt,3754.0
sugar,3081.0
onion,2819.0
pepper,2484.0
egg,2282.0
butter,2229.0
flour,1999.0
water,1753.0
olive oil,1456.0
milk,1456.0
